In [1]:
import numpy as np
import pandas as pd 
import os
import httplib2
from tqdm import tqdm
import shutil
import tensorflow as tf

In [3]:
print("pandas version: ", pd.__version__)
print("numpy version: ", np.__version__)
print("tensorflow version: ", tf.__version__)
print("httplib2 version: ", httplib2.__version__)

pandas version:  1.2.2
numpy version:  1.19.5
tensorflow version:  2.4.1
httplib2 version:  0.19.1


In [4]:
def download_image(url, guid):
    h = httplib2.Http('.cache')
    response, content = h.request(url)
    with open(f"images_add/{guid}.jpg", "wb") as f:
        f.write(content)

In [5]:
#фрейм с доп информацией
train_url_only = pd.read_csv("train_url_only.csv")

In [6]:
def download_with_df(df, from_, to_, x=None):
    if x==None:
        print("taf")
        for index, rows in tqdm(df[df['url'].isnull()==False].iloc[from_:to_].iterrows(), total=df[df['url'].isnull()==False].iloc[from_:to_].shape[0]):
            try:
                download_image(rows['url'], rows['guid'])
            except:
                print(rows['guid'])
    else:
        df = df.query(f"typology in {x}")
        for index, rows in tqdm(df[df['url'].isnull()==False].iloc[from_:to_].iterrows(), total=df[df['url'].isnull()==False].iloc[from_:to_].shape[0]):
            try:
                download_image(rows['url'], rows['guid'])
            except:
                print(rows['guid'])

In [7]:
download_with_df(train_url_only, 48677, 120000, x=['фотографии и негативы', 'предметы археологии', 'предметы нумизматики', 'документы', 'предметы прикладного искусства, быта и этнографии', 'предметы печатной продукции', 'графика', 'редкие книги'])

100%|██████████| 71323/71323 [8:59:00<00:00,  2.21it/s]   


In [8]:
#скачиваю с 0 до конца фрейма (по индексу дф) для категорий скульптура, оружие, предметы минералогической коллекции, предметы техники, прочие, живопись, предметы естественнонаучной коллекции    
download_with_df(train_url_only, 0, len(train_url_only), x=['скульптура', 'оружие', 'предметы минералогической коллекции', 'предметы техники', 'прочие', 'живопись', 'предметы естественнонаучной коллекции'])

Удаляю нулевые файлы

In [21]:
for filename in os.listdir("images_add"):
    if filename.endswith(".jpg"):    
        if os.path.getsize(f"images_add/{filename}")==0:
            os.remove(f"images_add/{filename}")

проверка на битые файлы

In [2]:
for filename in os.listdir("images_add"):
    if filename.endswith(".jpg"):
        try:
            img = tf.keras.preprocessing.image.load_img(f"images_add/{filename}")
        except:
            print(f"images_add/{filename}")
            shutil.move(f"images_add/{filename}", 'trash_images')

/home/eugene/anaconda3/lib/python3.8/site-packages/PIL/JpegImagePlugin.py:791: UserWarning: Image appears to be a malformed MPO file, it will be interpreted as a base JPEG file
  warnings.warn(
/home/eugene/anaconda3/lib/python3.8/site-packages/PIL/Image.py:2834: DecompressionBombWarning: Image size (143075400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/eugene/anaconda3/lib/python3.8/site-packages/PIL/Image.py:2834: DecompressionBombWarning: Image size (91203840 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


images_add/a8a34b22-cf29-4987-92dc-9615aa63c0d3.jpg


/home/eugene/anaconda3/lib/python3.8/site-packages/PIL/Image.py:2834: DecompressionBombWarning: Image size (124029123 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/eugene/anaconda3/lib/python3.8/site-packages/PIL/Image.py:2834: DecompressionBombWarning: Image size (91630500 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/eugene/anaconda3/lib/python3.8/site-packages/PIL/Image.py:2834: DecompressionBombWarning: Image size (97173120 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/eugene/anaconda3/lib/python3.8/site-packages/PIL/Image.py:2834: DecompressionBombWarning: Image size (172962790 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


images_add/d1649c1a-add1-4137-b680-ccfa8f480ded.jpg


/home/eugene/anaconda3/lib/python3.8/site-packages/PIL/Image.py:2834: DecompressionBombWarning: Image size (124851632 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/eugene/anaconda3/lib/python3.8/site-packages/PIL/Image.py:2834: DecompressionBombWarning: Image size (97667360 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


перемещение доп фото в основной каталог 

In [3]:
for filename in os.listdir("images_add"):
    shutil.move(f"images_add/{filename}", 'images')

In [8]:
#Итого: 141467 изображений